# Codename Clue Generator - Semantic Similarity

The goal of this test is to recommend clues for codename games given a current set of available cards and a word bank. The clues will be scored by their semantic similarity to the current cards:

- Positive points for cards belonging to the current team
- Negative points for cards belonging to the other team, neutral and double agent card



## Decisions to Make

- Choose embedding
  - Rolling with Wikipedia500 pre-trained [embeddings from TFHub](https://tfhub.dev/google/Wiki-words-500-with-normalization/2) for now
  - May want to swap to a different set at some point
- Decide on word bank
  - Remove stop words, articles (TODO, only currently removing single letter words)
  - What corpus should use for this?
    - Went with corpus generated from wikipedia and hosted on Kaggle: [English Word Frequency](https://www.kaggle.com/rtatman/english-word-frequency)
  - Top N english words? How do we decide on N?
    - Want enough variety, but need to test runtime of different values since calculating similarity vectors could be expensive at higher N, want to run fairly quickly but still have good enough clue quality
- Decide on similarity metric
  - Embedding has 500 dimensions. Going with cosine similarity which will hopefully help a little with high dimensionality over euclidean
- Decide on weights for scores
  - How highly do we negatively weight other team vs neutral vs double agent? 
- Decide how to select clues it applies to?
  - This will be done by taking the similarity scores for your team compared to the words in the word bank
  - We need a cutoff in terms of relative similarity scores (kind of like the elbow method in clustering) or absolute similarity 
  - How do we decide what the threshold is? No common metrics other than manual inspection?
- How much do we need optimization?
  - Caching scores for words and re-running for later clues in the same game
  - Caching word bank vectors
  - Normally, coming up with clues takes a long time so I don't think sub-second response time is necessary. May be able to get away with unoptimized code



## Library Installation and Loading

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install tensorflow_hub
!pip install pandas

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from random import shuffle
from google.colab import data_table  # This is just for viewing results more easily
data_table.enable_dataframe_formatter()

## Data Preparation Utilities

In [ ]:
def get_similarities(w1, w2):
    normalize_w1 = tf.nn.l2_normalize(w1,1)        
    normalize_w2 = tf.nn.l2_normalize(w2,1)
    distance = tf.matmul(normalize_w1, normalize_w2, transpose_b=True)
    return (distance + 1) / 2

def get_word_to_bank_similarities(words, word_bank_vector):
  words_vector = embed(words)
  return get_similarities(words_vector, word_bank_vector)

def get_example_board(codenames_words):
  words = codenames_words.copy()
  shuffle(words)
  return [
      words[0:9],
      words[9:17],
      words[17:24],
      [words[24]]
  ]

## Load Words and Initialize Parameters

In [ ]:
# PARAMS

top_n_words = 1000000

num_clues = 100

clue_similarity_threshold = 0.7

team_score_modifier = 10
enemy_score_modifier = -2
neutral_score_modifier = -1
double_agent_score_modifier = -5


In [ ]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/Wiki-words-500-with-normalization/2")

In [ ]:
# word_bank = pd.read_csv("unigram_freq.csv")
# word_bank = word_bank.loc[word_bank['word'].apply(lambda x: len(str(x))) > 1]
# word_bank = word_bank.iloc[0:min(word_bank.shape[0], top_n_words),:]
# word_bank.shape

with open('word_bank.txt', 'r') as f:
  word_bank = f.readlines()

word_bank = [w.strip() for w in word_bank if len(str(w)) > 1]
word_bank = word_bank[0:min(len(word_bank), top_n_words)]

In [ ]:
# with open('word_bank.txt', 'w') as f:
#   for word in word_bank['word'].values:
#     f.write('{}\n'.format(word))

In [ ]:
word_bank_all_words = word_bank
word_bank_vector = embed(word_bank_all_words)

In [ ]:
with open('codename_words.txt', 'r') as f:
  uppercase_words = f.readlines()

lowercase_words = [w.lower().strip() for w in uppercase_words]

## Clue Generator

In [ ]:
def censor_team_words_and_sum(team_scores, word_bank, team_words, game_words, clue_similarity_threshold):
  team_scores_filtered = []
  relevant_words = []
  clue_lengths = []
  for i in range(team_scores.shape[1]):
    score, clue_length, relevant_words_row = censor_row(team_scores[:, i], word_bank[i], game_words, team_words, clue_similarity_threshold)

    team_scores_filtered.append(score)
    relevant_words.append(relevant_words_row)
    clue_lengths.append(clue_length)

  return team_scores_filtered, clue_lengths, tuple(relevant_words)

def censor_row(row_scores, clue_word, game_words, team_words, clue_similarity_threshold):

  # TODO:
  # Censoring based on having subwords? This is when part or all of your clue exists within the cards on the board's words
  # Example: Submarine on the board, clue is sub or mariner

  sum = 0
  relevant_words = []
  clue_len = 0

  if clue_word in game_words:
    return sum, clue_len, relevant_words
  
  for relevant_word, score in zip(team_words, row_scores):
      if score >= clue_similarity_threshold: 

        sum += score
        relevant_words.append(relevant_word)
  
  clue_len = len(relevant_words)
  return sum, clue_len, relevant_words


In [ ]:
def generate_clues(num_clues, clue_similarity_threshold, word_bank, word_bank_vector, team_words, enemy_words, neutral_words, double_agent_words):

  game_words = team_words + enemy_words + neutral_words + double_agent_words

  team_scores = get_word_to_bank_similarities(team_words, word_bank_vector)

  team_scores_filtered, clue_lengths, relevant_words = censor_team_words_and_sum(team_scores.numpy(), word_bank, team_words, game_words, clue_similarity_threshold)

  team_scores = tf.convert_to_tensor(team_scores_filtered) * team_score_modifier
 

  enemy_scores = get_word_to_bank_similarities(enemy_words, word_bank_vector) * enemy_score_modifier
  neutral_scores = get_word_to_bank_similarities(neutral_words, word_bank_vector) * neutral_score_modifier
  double_agent_scores = get_word_to_bank_similarities(double_agent_words, word_bank_vector) * double_agent_score_modifier

  team_scores = tf.cast(tf.reshape(team_scores, (1, enemy_scores.shape[1])), tf.float32)

  all_scores = tf.concat([team_scores, enemy_scores, neutral_scores, double_agent_scores], 0)

  combined_scores = tf.reduce_sum(tf.transpose(all_scores), 1).numpy()

  recommendations_df = pd.DataFrame({
      'clue': word_bank,
      'clue_length': clue_lengths,
      'scores': combined_scores,
      'relevant_words': relevant_words
  })

  recommendations_df.sort_values(['scores'], ascending=False, inplace=True)
  # final_recommendations = recommendations_df
  final_recommendations = recommendations_df.iloc[0: num_clues]

  return team_scores, final_recommendations

In [ ]:
game_words = get_example_board(lowercase_words)
game_words

[['africa',
  'canada',
  'bridge',
  'ketchup',
  'platypus',
  'pie',
  'tail',
  'plastic',
  'fighter'],
 ['mint', 'scorpion', 'apple', 'charge', 'palm', 'bug', 'rock', 'fair'],
 ['bottle', 'moon', 'pan', 'police', 'snowman', 'berlin', 'root'],
 ['novel']]

In [ ]:
raw_scores, results = generate_clues(num_clues, clue_similarity_threshold, word_bank, word_bank_vector, *game_words)

In [ ]:
results['clue_str'] = results['relevant_words'].apply(lambda x: str(x))
results.groupby('clue_str').head(5).reset_index(drop=True)

,clue,clue_length,scores,relevant_words,clue_str
0,tins,3,6.910014,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
1,popcorn,3,6.734835,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
2,frosting,3,6.635225,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
3,toaster,3,6.534274,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
4,stuffing,3,6.457475,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
5,fillet,3,6.044208,"[ketchup, pie, tail]","['ketchup', 'pie', 'tail']"
6,goose,3,5.489191,"[platypus, pie, tail]","['platypus', 'pie', 'tail']"
7,rabbit,3,4.795670,"[platypus, pie, tail]","['platypus', 'pie', 'tail']"
8,stick,3,4.703341,"[pie, tail, plastic]","['pie', 'tail', 'plastic']"
9,shtml,2,2.584478,"[africa, canada]","['africa', 'canada']"


In [ ]:
results.head(50)

,clue,clue_length,scores,relevant_words,clue_str
25314,tins,3,6.910014,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
14106,popcorn,3,6.734835,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
36937,frosting,3,6.635225,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
17042,toaster,3,6.534274,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
18889,stuffing,3,6.457475,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
6187,oven,3,6.389229,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
12116,jelly,3,6.342927,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
6031,grill,3,6.332379,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
6717,paste,3,6.279557,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"
4843,candy,3,6.212758,"[ketchup, pie, plastic]","['ketchup', 'pie', 'plastic']"


## Summary

Initial results seem ok without a ton of fine tuning, provides at least one "reasonable" clue (from my subjective evaluation) per sampled game. 

Needs more research at lower numbers of available cards. Do the penalties override the signal gained from trying to guess the last 1-2 cards? Do the positive scores need to be scaled up as there are few left? What does that relationship look like? 

Overall, more parameter exploration needs to be tested along with a more formal way to judge relevance. 

### Failure Modes

- Words that have multiple meanings get diluted by single vector repesentations. 
- Words that have meanings outside of a formal information site like Wikipedia that the embeddings were trained on

In [ ]:
|